In [2]:
## Install necessary libraries
%pip install -q -r ../requirements.txt


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# configure tensorflow for M1 max with 32 GPU cores


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
# Load ResNet50 model with mixed precision for better performance on Metal
policy = tf.keras.mixed_precision.Policy('mixed_float16')
tf.keras.mixed_precision.set_global_policy(policy)

#  Fix SSL certificate verification issue
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Load model with Metal GPU optimization
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)
print("Model loaded successfully with Metal GPU support")


2025-05-28 00:19:33.183894: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-05-28 00:19:33.183945: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-05-28 00:19:33.183952: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
I0000 00:00:1748387973.184024 3770965 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1748387973.184098 3770965 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model loaded successfully with Metal GPU support


In [28]:
import tensorflow_datasets as tfds
import tensorflow as tf

# Set batch size and image size
BATCH_SIZE = 32
IMAGE_SIZE = 224
AUTOTUNE = tf.data.AUTOTUNE

def preprocess_data(data):
    """Preprocess ImageNet data for ResNet50"""
    image = tf.cast(data['image'], tf.float32)
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = tf.keras.applications.resnet50.preprocess_input(image)
    label = data['label']
    return image, label

# Load ImageNet test dataset - 13 minutes to load data
dataset, info = tfds.load(
    'imagenet2012',
    shuffle_files=False,
    with_info=True,
    data_dir='../datasets'  # Using your local ImageNet data
)

# Update to use the validation split instead of the test split
# validation_dataset = dataset['validation'].map(preprocess_data, num_parallel_calls=AUTOTUNE)
# validation_dataset = validation_dataset.batch(BATCH_SIZE)
# validation_dataset = validation_dataset.prefetch(AUTOTUNE)

# Replace test_dataset with validation_dataset in subsequent code

In [ ]:
## Load test dataset
test_dataset, info = tfds.load(
    'imagenet_v2',
    shuffle_files=False,
    with_info=True
)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:02<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:02<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:02<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:03<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:06<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:07<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:09<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:09<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:10<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:10<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:11<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:12<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:12<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:13<?, ? url/s]

Dl Completed...:   0%|          | 0/1 [00:14<?, ? url/s]

Dl Completed...:   0%|          

In [29]:
# Evaluation
# Configure the dataset for performance
test_dataset = validation_dataset

# test_dataset
# Evaluate the model
# model.summary()
model.compile(
	optimizer='adam',
	loss='sparse_categorical_crossentropy',
	metrics=['accuracy']
)
